In [1]:
from llama_index.llms.ollama import Ollama
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import re
import pickle

c:\Users\haris\miniconda3\envs\env_langchain1\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class SentimentModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, dropout_rate=0.1, l2_lambda=0.001, num_classes=3):
        super(SentimentModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(embedding_dim * max_length, 128)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.flatten(x)
        x = nn.ReLU()(self.fc1(x))
        x = self.dropout1(x)
        x = nn.ReLU()(self.fc2(x))
        x = self.dropout2(x)
        x = nn.ReLU()(self.fc3(x))
        x = self.fc4(x)
        return x

In [3]:
with open(r"D:\Senti_chatbot\notebook\vocab.pkl", 'rb') as f:
    vocab = pickle.load(f)
max_length = 50

In [ ]:
def tokenize(text):
    return text.split()

def text_to_sequence(text, vocab, max_length):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text) 
    tokens = tokenize(text)
    sequence = [vocab.get(token, vocab["<unk>"]) for token in tokens]
    return sequence[:max_length] + [vocab["<pad>"]] * (max_length - len(sequence))

# Load the trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentimentModel(vocab_size=len(vocab))
model.load_state_dict(torch.load(r'D:\Senti_chatbot\notebook\sentiment_model.pth', map_location=device))
model = model.to(device)
model.eval()

# Define the label mapping
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

C:\Users\haris\AppData\Local\Temp\ipykernel_22888\3130546986.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r'D:\Senti_chatbot\noteboo

In [5]:
def predict_sentiment(sentence):
    sequence = text_to_sequence(sentence, vocab, max_length)
    sequence_tensor = torch.tensor([sequence]).to(device)
    with torch.no_grad():
        output = model(sequence_tensor)
        predicted_label = output.argmax(1).item()
    return label_map[predicted_label]

In [26]:
def fetch_llama_response(prompt):
        # Initialize the Ollama model
    llama_model = Ollama(base_url="http://127.0.0.1:11436", model="llama3.2:1b")

        # Ensure the prompt is passed as a string
    response_stream = llama_model.complete(prompt=prompt, max_tokens=100)
        # Handle different response formats
    return response_stream
    if isinstance(response_stream, str):
        return response_stream.strip()
    elif hasattr(response_stream, "__iter__"):
        response_text = "".join(chunk["text"] for chunk in response_stream if isinstance(chunk, dict) and "text" in chunk)
        return response_text.strip()
    else:
        return "Unexpected response format from Llama model."

In [27]:
def chatbot():
    print("Welcome to the Sentiment-Driven Chatbot! Type 'exit' to end the chat.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break
        
        # Predict sentiment
        sentiment = predict_sentiment(user_input)
        print(f"[Sentiment Detected: {sentiment}]")
        
        # Enhance prompt and fetch bot response
        enhanced_prompt = f"The user's sentiment is {sentiment}. Respond empathetically to: {user_input}"
        bot_response = fetch_llama_response(enhanced_prompt)
        print(f"Chatbot: {bot_response}")

# Start Chatbot
chatbot()

Welcome to the Sentiment-Driven Chatbot! Type 'exit' to end the chat.
[Sentiment Detected: Neutral]


ReadTimeout: timed out